In [1]:
#Installing and imporing packages and moduls
!pip install arch 
!pip install yfinance
!pip install pmdarima


import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import scipy
import yfinance as yf
from statsmodels.tsa.arima.model import ARIMA 
from statsmodels.graphics.api import qqplot
from statsmodels.stats.diagnostic import acorr_ljungbox as lb_test
from statsmodels.tsa.stattools import adfuller
from pmdarima.arima import auto_arima
from scipy import stats
from arch import arch_model
from datetime import timedelta
from itertools import product
from math import sqrt
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings("ignore")

import matplotlib.lines as mlines
import matplotlib.pyplot as plt
%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 903 kB 5.2 MB/s 
     |████████████████████████████████| 9.8 MB 49.7 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 7.5 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that 

In [2]:
#Getting data from Yahoo!Finance


bitcoin = yf.Ticker("BTC-USD")
data = bitcoin.history(interval='1d', start='2018-06-30', end='2022-06-30')
print(data)

                    Open          High           Low         Close  \
Date                                                                 
2018-06-30   6214.220215   6465.509766   6214.220215   6404.000000   
2018-07-01   6411.680176   6432.850098   6289.290039   6385.819824   
2018-07-02   6380.379883   6683.859863   6305.700195   6614.180176   
2018-07-03   6596.660156   6671.370117   6447.750000   6529.589844   
2018-07-04   6550.870117   6771.919922   6450.459961   6597.549805   
...                  ...           ...           ...           ...   
2022-06-26  21496.494141  21783.724609  21016.269531  21027.294922   
2022-06-27  21028.238281  21478.089844  20620.199219  20735.478516   
2022-06-28  20731.544922  21164.423828  20228.812500  20280.634766   
2022-06-29  20281.169922  20364.156250  19937.791016  20104.023438   
2022-06-30  20108.312500  20141.160156  18729.656250  19784.726562   

                 Volume  Dividends  Stock Splits  
Date                                  

In [3]:
#Calculate returns

data['log_price'] = np.log(data['Close'])
data['return'] = data['log_price'] - data['log_price'].shift(1)

#Data+1, so we can perform 
return_data = data['return']+1 
return_data = return_data.dropna()
print(return_data)


Date
2018-07-01    0.997157
2018-07-02    1.035136
2018-07-03    0.987128
2018-07-04    1.010354
2018-07-05    1.006284
                ...   
2022-06-26    0.977660
2022-06-27    0.986025
2022-06-28    0.977820
2022-06-29    0.991253
2022-06-30    0.983990
Name: return, Length: 1461, dtype: float64


In [4]:
#Stationarity test

#Stationarity in variance test (Box-Cox)
  #H0: Data is not stasioner
  #H1: Data is stasioner
transformed_data, best_lambda = scipy.stats.boxcox(return_data)
best_lambda = round(best_lambda, 0)
print(best_lambda)

return_data = pd.Series(return_data)
transformed_data = return_data.pow(best_lambda)
print(transformed_data)

3.0
Date
2018-07-01    0.991495
2018-07-02    1.109155
2018-07-03    0.961880
2018-07-04    1.031385
2018-07-05    1.018971
                ...   
2022-06-26    0.934465
2022-06-27    0.958658
2022-06-28    0.934926
2022-06-29    0.973989
2022-06-30    0.952736
Name: return, Length: 1461, dtype: float64


In [5]:
#Stationerity in mean (Augmented Dickey Fuller)
  #H0: Data is not stationer
  #H1: Data is stasioner
def adf_test(trans):
  result = adfuller(trans)
  labels = ['ADF Test Statistics', 'p-value', 'Lag Used', 'Number of Observations Used']
  for value, label in zip(result,labels):
    print(label+' : '+str(value))
  if result[1] <= 0.05:
    print("Null hypothesis is rejected. Data is stationer")
  else:
    print("Null hypothesis is accepted. Data is not stationer")


adf_test(transformed_data)
transformed_data = pd.Series(transformed_data)

ADF Test Statistics : -17.91803256521895
p-value : 2.9130880588388586e-30
Lag Used : 3
Number of Observations Used : 1457
Null hypothesis is rejected. Data is stationer


In [ ]:
#diferentiating data
transformed_data = pd.Series(transformed_data)
transformed_data= transformed_data-transformed_data.shift(1)

In [ ]:
adf_test(transformed_data.dropna())

ADF Test Statistics : -10.185768232248643
p-value : 6.515041803246412e-18
Lag Used : 7
Number of Observations Used : 176
Null hypothesis is rejected. Data is stationer


In [6]:
#Auto ARIMA
data_arima = auto_arima(transformed_data, trace=True)

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=-2235.325, Time=3.86 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-2234.778, Time=0.24 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=-2236.954, Time=0.26 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-2236.645, Time=0.35 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=4185.741, Time=0.03 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=-2237.119, Time=2.91 sec
 ARIMA(3,0,0)(0,0,0)[0] intercept   : AIC=-2235.119, Time=0.74 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=-2235.119, Time=0.94 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=-2238.280, Time=4.59 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=-2234.900, Time=2.74 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=-2236.891, Time=0.98 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=inf, Time=2.10 sec

Best model:  ARIMA(1,0,1)(0,0,0)[0] intercept
Total fit time: 19.831 seconds


In [7]:
#Heteroscedasticiry in ARIMA residual (LM Test)
  #H0: Data is not stationer
  #H1: Data is stasioner
from statsmodels.stats.diagnostic import het_arch

def LM_test(residuals):
  result = het_arch(residuals)
  labels = ['LM Test Statistics', 'LM p-value', 'F test Statistics', 'F p-value']
  for value, label in zip(result,labels):
    print(label+' : '+str(value))
  if result[1] <= 0.05:
    print("Null hypothesis is rejected. Residuals isn't homoscedastic")
  else:
    print("Failed to reject the null hypothesis. Residuals is homoscedastic")

In [8]:
#ARIMA(0,0,2)
arima_002 = ARIMA(transformed_data.dropna(), order=(0,0,2))
arima_002 = arima_002.fit()
print(arima_002.summary(), "\n")
arima_002_resid = arima_002.resid
LM_test(arima_002_resid)
print("\n")

#ARIMA(1,0,0)
arima_100 = ARIMA(transformed_data.dropna(), order=(1,0,0))
arima_100 = arima_100.fit()
print(arima_100.summary(), "\n")
arima_100_resid = arima_100.resid
LM_test(arima_100_resid)
print("\n")

#ARIMA(0,0,1)
arima_001 = ARIMA(transformed_data.dropna(), order=(0,0,1))
arima_001 = arima_001.fit()
print(arima_001.summary(), "\n")
arima_001_resid = arima_001.resid
LM_test(arima_001_resid)
print("\n")

#ARIMA(2,0,0)
arima_200 = ARIMA(transformed_data.dropna(), order=(2,0,0))
arima_200 = arima_200.fit()
print(arima_200.summary(), "\n")
arima_200_resid = arima_200.resid
LM_test(arima_200_resid)
print("\n")

#ARIMA(3,0,0)
arima_300 = ARIMA(transformed_data.dropna(), order=(3,0,0))
arima_300 = arima_300.fit()
print(arima_300.summary(), "\n")
arima_300_resid = arima_300.resid
LM_test(arima_300_resid)
print("\n")

#ARIMA(2,0,1)
arima_201 = ARIMA(transformed_data.dropna(), order=(2,0,1))
arima_201 = arima_201.fit()
print(arima_201.summary(), "\n")
arima_201_resid = arima_201.resid
LM_test(arima_201_resid)
print("\n")

#ARIMA(1,0,1)
arima_101 = ARIMA(transformed_data.dropna(), order=(1,0,1))
arima_101 = arima_101.fit()
print(arima_101.summary(), "\n")
arima_101_resid = arima_101.resid
LM_test(arima_101_resid)
print("\n")

#ARIMA(1,0,2)
arima_102 = ARIMA(transformed_data.dropna(), order=(1,0,2))
arima_102 = arima_102.fit()
print(arima_102.summary(), "\n")
arima_102_resid = arima_102.resid
LM_test(arima_102_resid)
print("\n")

#ARIMA(2,0,2)
arima_202 = ARIMA(transformed_data.dropna(), order=(2,0,2))
arima_202 = arima_202.fit()
print(arima_202.summary(), "\n")
arima_202_resid = arima_202.resid
LM_test(arima_202_resid)
print("\n")

                               SARIMAX Results                                
Dep. Variable:                 return   No. Observations:                 1461
Model:                 ARIMA(0, 0, 2)   Log Likelihood                1122.445
Date:                Mon, 04 Jul 2022   AIC                          -2236.890
Time:                        01:06:24   BIC                          -2215.743
Sample:                    07-01-2018   HQIC                         -2229.002
                         - 06-30-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0067      0.003    344.918      0.000       1.001       1.012
ma.L1         -0.0518      0.022     -2.387      0.017      -0.094      -0.009
ma.L2          0.0370      0.024      1.572      0.1

In [20]:
#ARIMA(1,0,1)-GARCH(1,0)
GARCH_10 = arch_model(arima_101_resid, vol='GARCH', p=1, q=0)
GARCH_10 = GARCH_10.fit()
print(GARCH_10.summary(), "\n")

#ARIMA(1,0,1)-GARCH(2,0)
GARCH_20 = arch_model(arima_101_resid, vol='GARCH', p=2, q=0)
GARCH_20 = GARCH_20.fit()
print(GARCH_20.summary(), "\n")

#ARIMA(1,0,1)-GARCH(1,1)
ARCH_GARCH_11 = arch_model(arima_101_resid, vol='GARCH', p=1, q=1)
ARCH_GARCH_11 = ARCH_GARCH_11.fit()
print(ARCH_GARCH_11.summary(), "\n")
garch11_res = ARCH_GARCH_11.resid

#ARIMA(1,0,1)-GARCH(2,1)
ARCH_GARCH_21 = arch_model(arima_101_resid, vol='GARCH', p=2, q=1)
ARCH_GARCH_21 = ARCH_GARCH_21.fit()
print(ARCH_GARCH_21.summary(), "\n")

#ARIMA(1,0,1)-GARCH(2,2)
ARCH_GARCH_22 = arch_model(arima_101_resid, vol='GARCH', p=2, q=2)
ARCH_GARCH_22 = ARCH_GARCH_22.fit()
print(ARCH_GARCH_22.summary(), "\n")

Iteration:      1,   Func. Count:      5,   Neg. LLF: -1006.1560338793346
Iteration:      2,   Func. Count:     15,   Neg. LLF: -1006.1959043016577
Iteration:      3,   Func. Count:     24,   Neg. LLF: -1113.8483493560952
Iteration:      4,   Func. Count:     29,   Neg. LLF: -1123.0975189017538
Iteration:      5,   Func. Count:     35,   Neg. LLF: -1124.3971227644397
Iteration:      6,   Func. Count:     43,   Neg. LLF: -1127.825436095752
Iteration:      7,   Func. Count:     48,   Neg. LLF: -1127.8534742146735
Iteration:      8,   Func. Count:     53,   Neg. LLF: -1127.8558242213298
Iteration:      9,   Func. Count:     58,   Neg. LLF: -1127.856449453731
Iteration:     10,   Func. Count:     63,   Neg. LLF: -1127.85689485476
Iteration:     11,   Func. Count:     68,   Neg. LLF: -1127.8569121856765
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -1127.8569127273513
            Iterations: 11
            Function evaluations: 69
            Gra

In [21]:
LM_test(garch11_res)

LM Test Statistics : 54.30666711462051
LM p-value : 4.254034447763812e-08
F test Statistics : 5.599053049354772
F p-value : 3.049376473625254e-08
Null hypothesis is rejected. Residuals isn't homoscedastic


In [26]:
#ARIMA(1,0,1)-EGARCH(1,1)
EGARCH_11 = arch_model(arima_101_resid, vol='EGARCH', p=1, q=1)
EGARCH_11 = EGARCH_11.fit()
print(EGARCH_11.summary(), "\n")

#ARIMA(1,0,1)-EGARCH(1,2)
EGARCH_12 = arch_model(arima_101_resid, vol='EGARCH', p=1, q=2)
EGARCH_12 = EGARCH_12.fit()
print(EGARCH_12.summary(), "\n")

#ARIMA(1,0,1)-GJR GARCH(1,2)
GJR_GARCH_111 = arch_model(arima_101_resid, vol='GARCH', p=1, o=1, q=1)
GJR_GARCH_111 = GJR_GARCH_111.fit()
print(GJR_GARCH_111.summary(), "\n")

#ARIMA(1,0,1)-GJR GARCH(1,2)
GJR_GARCH_112 = arch_model(arima_101_resid, vol='GARCH', p=1, o=1, q=2)
GJR_GARCH_112 = GJR_GARCH_112.fit()
print(GJR_GARCH_112.summary(), "\n")

#ARIMA(1,0,1)-GJR GARCH(1,2)
GJR_GARCH_121 = arch_model(arima_101_resid, vol='GARCH', p=1, o=2, q=1)
GJR_GARCH_121 = GJR_GARCH_121.fit()
print(GJR_GARCH_121.summary(), "\n")

Iteration:      1,   Func. Count:      6,   Neg. LLF: -1161.5733336983137
Iteration:      2,   Func. Count:     16,   Neg. LLF: -1163.801552147041
Iteration:      3,   Func. Count:     28,   Neg. LLF: -1164.635990903073
Iteration:      4,   Func. Count:     38,   Neg. LLF: -1166.1508887591222
Iteration:      5,   Func. Count:     45,   Neg. LLF: -1166.5283923982392
Iteration:      6,   Func. Count:     51,   Neg. LLF: -1166.6797484681313
Iteration:      7,   Func. Count:     57,   Neg. LLF: -1166.712165738795
Iteration:      8,   Func. Count:     63,   Neg. LLF: -1166.7132752325222
Iteration:      9,   Func. Count:     69,   Neg. LLF: -1166.7135830982734
Iteration:     10,   Func. Count:     75,   Neg. LLF: -1166.7136254412733
Iteration:     11,   Func. Count:     81,   Neg. LLF: -1166.713652464144
Iteration:     12,   Func. Count:     88,   Neg. LLF: -1166.7136661331215
Iteration:     13,   Func. Count:     96,   Neg. LLF: -1166.7136688802575
Optimization terminated successfully.    (